In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import os

In [2]:
data_dir = "./Big Star Collectibles"

In [3]:
files = os.listdir(data_dir)
file_texts = []
for file in files:
    with open(f"{data_dir}/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=128, chunk_overlap=32, # this is the critical line
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text,metadata={ 
                    "doc_title": file.split(".")[0],
                    "chunk_num": i})) 

Created a chunk of size 139, which is longer than the specified 128
Created a chunk of size 151, which is longer than the specified 128
Created a chunk of size 151, which is longer than the specified 128
Created a chunk of size 139, which is longer than the specified 128
Created a chunk of size 130, which is longer than the specified 128
Created a chunk of size 188, which is longer than the specified 128
Created a chunk of size 130, which is longer than the specified 128


In [4]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings() # embed your data

/tmp/ipykernel_14526/3428753526.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings() # embed your data
/tmp/ipykernel_14526/3428753526.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() # embed your data
/workspaces/advanced-rag-applications-with-vector-databases-3886256/ch1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please upd

In [6]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
# store the embedded data into a vector database
vector_store = FAISS.from_documents(
    file_texts, 
    embedding=embeddings
)

In [8]:
vector_store.save_local("faiss_index_backup") # save the vector database for later use
vector_store